In [1]:
from sympy import *  
init_printing() 
import numpy as np

In [2]:
A = Matrix([[1, 2, 3], 
            [4, 5, 6], 
            [7,8,9]])

A

⎡1  2  3⎤
⎢       ⎥
⎢4  5  6⎥
⎢       ⎥
⎣7  8  9⎦

In [3]:
# To find the transpose:

A.T

⎡1  4  7⎤
⎢       ⎥
⎢2  5  8⎥
⎢       ⎥
⎣3  6  9⎦

In [4]:
# To construct an identity matrix:

display(eye(2))

display(eye(4))

⎡1  0⎤
⎢    ⎥
⎣0  1⎦

⎡1  0  0  0⎤
⎢          ⎥
⎢0  1  0  0⎥
⎢          ⎥
⎢0  0  1  0⎥
⎢          ⎥
⎣0  0  0  1⎦

In [5]:
# To take the determinant:

A.det()

In [6]:
x, alpha, beta = symbols("x alpha beta")

A = Matrix([[x, 1, 0, 0, 1],
       [1, x, 1, 0, 0],
       [0, 1, x, 1, 0],
       [0, 0, 1, x, 1],
       [1, 0, 0, 1, x]]
      )

A

⎡x  1  0  0  1⎤
⎢             ⎥
⎢1  x  1  0  0⎥
⎢             ⎥
⎢0  1  x  1  0⎥
⎢             ⎥
⎢0  0  1  x  1⎥
⎢             ⎥
⎣1  0  0  1  x⎦

In [7]:
A.det()

# --------------------------------------------------------------------------------

In [8]:
s = Function('s')
d = Function('d')
phi = Function('phi')
i, j, a, b, a1, b1 = symbols("i, j, a, b, a1, b1")

In [13]:
from sympy.physics.quantum import Bra, Ket, TensorProduct

Bra(phi(i, j))*Ket(phi(a, b) - phi(a1, b1)) 


In [24]:
def dirac_expand_1(bra, bra1, ket, ket1):
    if bra1 == 1:
        return Bra(bra)*Ket(ket) + Bra(bra)*Ket(ket1)
    else:
        return Bra(bra)*Ket(ket) + Bra(bra1)*Ket(ket) + Bra(bra)*Ket(ket1) + Bra(bra1)*Ket(ket1)


display(dirac_expand_1(phi(i, j), 1, phi(a, b), -phi(a1, b1)))


def dirac_to_matrix(var1, var2, var3, var4, var5 = None, var6 = None):
    if var5 == None and var6 == None:
        return Matrix([
            [(var1)*(var3), (var1)*(var4)],
            [(var2)*(var3), (var2)*(var4)]
                                        ])
    else:
        return Matrix([
            [var1*var4, var1*var5, var1*var6],
            [var2*var4, var2*var5, var2*var6],
            [var3*var4, var3*var5, var3*var6]
                                            ])


dirac_to_matrix(phi(i), phi(j), phi(a), phi(b), phi(a1), phi(b1))

⎡φ(b)⋅φ(i)  φ(a₁)⋅φ(i)  φ(b₁)⋅φ(i)⎤
⎢                                 ⎥
⎢φ(b)⋅φ(j)  φ(a₁)⋅φ(j)  φ(b₁)⋅φ(j)⎥
⎢                                 ⎥
⎣φ(a)⋅φ(b)  φ(a)⋅φ(a₁)  φ(a)⋅φ(b₁)⎦

In [32]:
A1 = Matrix([
    [phi(i)*phi(a), phi(i)*phi(b)],
    [phi(j)*phi(a), phi(j)*phi(b)]
])

A2 = Matrix([
    [phi(i)*phi(a1), phi(i)*phi(b1)],
    [phi(j)*phi(a1), phi(j)*phi(b1)]
])

display(A1, A2)

⎡φ(a)⋅φ(i)  φ(b)⋅φ(i)⎤
⎢                    ⎥
⎣φ(a)⋅φ(j)  φ(b)⋅φ(j)⎦

⎡φ(a₁)⋅φ(i)  φ(b₁)⋅φ(i)⎤
⎢                      ⎥
⎣φ(a₁)⋅φ(j)  φ(b₁)⋅φ(j)⎦

In [33]:
n = Symbol("n")

    
def Matrix_transform_1(matrix, dim):
    if dim == 2:
        Matrix_1 = matrix.col(0).replace(phi, d)
        Matrix_2 = matrix.col(1).replace(phi, s)
        
        return Matrix([[Matrix_1, Matrix_2]])
    
    if dim == 3:
        Matrix_1 = matrix.col(0).replace(phi, d)
        Matrix_2 = matrix.col(1).replace(phi, s)
        Matrix_3 = matrix.col(2).replace(phi, s)
        return Matrix([[Matrix_1, Matrix_2, Matrix_3]])
    
    else:
        Matrix_1 = matrix.col(0).replace(phi, d)
        Matrix_2 = matrix.col(1).replace(phi, s)
        Matrix_3 = matrix.col(2).replace(phi, s)
        Matrix_4 = matrix.col(3).replace(phi, s)
        return Matrix([[Matrix_1, Matrix_2, Matrix_3, Matrix_4]])

def Matrix_transform_2(matrix, dim):
    if dim == 2:
        Matrix_1 = matrix.col(0).replace(phi, s)
        Matrix_2 = matrix.col(1).replace(phi, d)
        
        return Matrix([[Matrix_1, Matrix_2]])
    
    if dim == 3:
        Matrix_1 = matrix.col(0).replace(phi, s)
        Matrix_2 = matrix.col(1).replace(phi, d)
        Matrix_3 = matrix.col(2).replace(phi, s)
        return Matrix([[Matrix_1, Matrix_2, Matrix_3]])
    
    else:
        Matrix_1 = matrix.col(0).replace(phi, s)
        Matrix_2 = matrix.col(1).replace(phi, d)
        Matrix_3 = matrix.col(2).replace(phi, s)
        Matrix_4 = matrix.col(3).replace(phi, s)
        return Matrix([[Matrix_1, Matrix_2, Matrix_3, Matrix_4]])


def Matrix_transform_3(matrix, dim):
    
    
    if dim == 2:
        return 0
    
    if dim == 3:
        Matrix_1 = matrix.col(0).replace(phi, s)
        Matrix_2 = matrix.col(1).replace(phi, s)
        Matrix_3 = matrix.col(2).replace(phi, d)
        return Matrix([[Matrix_1, Matrix_2, Matrix_3]])
    
    else:
        Matrix_1 = matrix.col(0).replace(phi, s)
        Matrix_2 = matrix.col(1).replace(phi, s)
        Matrix_3 = matrix.col(2).replace(phi, d)
        Matrix_4 = matrix.col(3).replace(phi, s)
        return Matrix([[Matrix_1, Matrix_2, Matrix_3, Matrix_4]])

def Matrix_transform_4(matrix, dim):
    
    if dim == 2 or dim == 3:
        return 0
    
    else:
        Matrix_1 = matrix.col(0).replace(phi, s)
        Matrix_2 = matrix.col(1).replace(phi, s)
        Matrix_3 = matrix.col(2).replace(phi, s)
        Matrix_4 = matrix.col(3).replace(phi, d)
        return Matrix([[Matrix_1, Matrix_2, Matrix_3, Matrix_4]])


def matrix_transform(matrix, dim):
    if dim == 2:
        return TensorProduct(Matrix_transform_1(matrix, dim) + Matrix_transform_2(matrix, dim), eye(dim))
    if dim == 3:
        return display(TensorProduct(Matrix_transform_1(matrix, dim))) , display(TensorProduct(Matrix_transform_2(matrix, dim))) , display(TensorProduct(Matrix_transform_3(matrix, dim), eye(dim)))
    if dim == 4:
        return display(TensorProduct(Matrix_transform_1(matrix, dim), eye(dim))) , display(TensorProduct(Matrix_transform_2(matrix, dim), eye(dim))) , display(TensorProduct(Matrix_transform_3(matrix, dim), eye(dim))) , display(TensorProduct(Matrix_transform_4(matrix, dim), eye(dim)))


# R = simplify(Matrix_transform_1(B) + Matrix_transform_2(B))

a, b, c, e, i, j, k, l = symbols("a, b, c, e, i, j, k, l")
psi = Function('psi')

M = Matrix([[phi(a), phi(b), phi(c), phi(e)],
       [phi(a), phi(b), phi(c), phi(e)],
       [phi(i), phi(j), phi(k), phi(l)],
           [phi(a), phi(l), phi(k), phi(e)]
           ])

matrix_transform(M, 4)

⎡d(a)  s(b)  s(c)  s(e)⎤
⎢                      ⎥
⎢d(a)  s(b)  s(c)  s(e)⎥
⎢                      ⎥
⎢d(i)  s(j)  s(k)  s(l)⎥
⎢                      ⎥
⎣d(a)  s(l)  s(k)  s(e)⎦

⎡s(a)  d(b)  s(c)  s(e)⎤
⎢                      ⎥
⎢s(a)  d(b)  s(c)  s(e)⎥
⎢                      ⎥
⎢s(i)  d(j)  s(k)  s(l)⎥
⎢                      ⎥
⎣s(a)  d(l)  s(k)  s(e)⎦

⎡s(a)  s(b)  d(c)  s(e)⎤
⎢                      ⎥
⎢s(a)  s(b)  d(c)  s(e)⎥
⎢                      ⎥
⎢s(i)  s(j)  d(k)  s(l)⎥
⎢                      ⎥
⎣s(a)  s(l)  d(k)  s(e)⎦

⎡s(a)  s(b)  s(c)  d(e)⎤
⎢                      ⎥
⎢s(a)  s(b)  s(c)  d(e)⎥
⎢                      ⎥
⎢s(i)  s(j)  s(k)  d(l)⎥
⎢                      ⎥
⎣s(a)  s(l)  s(k)  d(e)⎦

(None, None, None, None)

In [37]:
SUCCESS = Matrix_transform(A1, 2)
maybe = Matrix_transform(A2, 2)

display(factor(SUCCESS), factor(maybe))

expand(det(factor(simplify(SUCCESS))) + det(factor(simplify(maybe))))


def determinant(matrix):
    return expand(det(factor(simplify(matrix))))

q = determinant(SUCCESS) + determinant(maybe)

q

⎡d(a)⋅d(i) + s(a)⋅s(i)  d(b)⋅d(i) + s(b)⋅s(i)⎤
⎢                                            ⎥
⎣d(a)⋅d(j) + s(a)⋅s(j)  d(b)⋅d(j) + s(b)⋅s(j)⎦

⎡d(a₁)⋅d(i) + s(a₁)⋅s(i)  d(b₁)⋅d(i) + s(b₁)⋅s(i)⎤
⎢                                                ⎥
⎣d(a₁)⋅d(j) + s(a₁)⋅s(j)  d(b₁)⋅d(j) + s(b₁)⋅s(j)⎦

Note: need to break it into two matrices to compute properly.

In [38]:
qr = q.replace(i, alpha).replace(j, beta).replace(a, 2*alpha).replace(b, beta).replace(a1, 2*beta).replace(b1, alpha)

simplify((1/sqrt(2))*qr)

In [44]:
S = (SUCCESS.replace(i, alpha).replace(j, beta).replace(a, 2*alpha).replace(b, beta).replace(a1, 2*beta).replace(b1, alpha))
m = (maybe.replace(i, alpha).replace(j, beta).replace(a, 2*alpha).replace(b, beta).replace(a1, 2*beta).replace(b1, alpha))

expand(det(S) + det(m))

def coupling_simplify(matrix):
    nreps = 2
    
    var = float(var)
    var1 = float(var1)
    initial = [ d(var)*d(var), s(var1)*s(var1)]
    final = [0, 1]

    for i in range(nreps):
        res = matrix.replace(initial[i], final[i])
        
    return res

coupling_simplify(qr)

UnboundLocalError: local variable 'var' referenced before assignment

Trying three electron systems...

In [ ]:
M = Matrix([
    [phi(alpha)*phi(alpha), phi(alpha)*phi(beta), phi(alpha)*phi(3*alpha)],
    [phi(2*beta)*phi(alpha), phi(2*beta)*phi(beta), phi(2*beta)*phi(3*alpha)],
    [phi(3*alpha)*phi(alpha), phi(3*alpha)*phi(beta), phi(3*alpha)*phi(3*alpha)]
])

display(M)

N1 = Matrix_transform_1(M, 3).replace(s(alpha)*s(beta), 0).replace(s(3*alpha)*s(beta), 0).replace(d(alpha)*d(2*beta), 0).replace(s(3*alpha)*s(2*beta), 0)# , \
N2 = Matrix_transform_2(M, 3).replace(s(alpha)*s(2*beta), 0).replace(d(alpha)*d(beta), 0).replace(s(3*alpha)*s(2*beta), 0).replace(d(3*alpha)*d(beta), 0) #, \
N3 = Matrix_transform_3(M, 3).replace(s(alpha)*s(2*beta), 0).replace(s(alpha)*s(beta), 0).replace(s(3*alpha)*s(beta), 0).replace(d(3*alpha)*d(2*beta), 0)


RF = TensorProduct(N1, eye(3)) + TensorProduct(N2, eye(3)) + TensorProduct(N3, eye(3))

expand(det(RF))

In [ ]:
omega = Symbol("Omega")

M = Matrix([
    [phi(a)*phi(a), phi(a)*phi(b), phi(a)*phi(omega)],
    [phi(2*b)*phi(a), phi(2*b)*phi(b), phi(2*b)*phi(omega)],
    [phi(omega)*phi(a), phi(omega)*phi(b), phi(omega)*phi(omega)]
])

M

Q = Matrix_transform_1(M, 3)
X = Matrix_transform_2(M, 3)
Y = Matrix_transform_3(M, 3)

In [ ]:
RF = ((TensorProduct(Q, eye(3)) + TensorProduct(X, eye(3)) + TensorProduct(Y, eye(3))))

RF

In [ ]:
M = Matrix([
    [phi(a)*phi(i), phi(a)*phi(j), phi(a)*phi(k)],
    [phi(b)*phi(i), phi(b)*phi(j), phi(b)*phi(k)],
    [phi(c)*phi(i), phi(c)*phi(j), phi(c)*phi(k)]
])

display(M)

a, b, c, i, j, k = symbols("a, b, c, i, j, k")

In [ ]:
Matrix_transform(M, 3)

Change definition of matrix transform. Have to do each separately. (?) What if you don't have actual values for the electrons?

In [ ]:
det(Matrix_transform(M, 3))